In [ ]:
import glob

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from concurrent.futures import ThreadPoolExecutor
import requests

from osgeo import gdal

gdal.SetConfigOption('GDAL_PDF_LAYERS', 'Map_Frame')


In [ ]:
# Coordinates of upper left corner, add -0.125N, 0.125W to get lower right
map_corners = [
    (42.375, -75.125, 'NY_Treadwell_20190925_TM_geo.pdf'),
    (42.375, -75.0, 'NY_Delhi_20190919_TM_geo.pdf'),
    (42.375, -74.875, 'NY_Bloomville_20190919_TM_geo.pdf'),
    (42.375, -74.75, 'NY_Hobart_20190919_TM_geo.pdf'),
    (42.375, -74.625, 'NY_Roxbury_20190919_TM_geo.pdf'),
    (42.375, -74.5, 'NY_Prattsville_20190919_TM_geo.pdf'),
    (42.375, -74.375, 'NY_Ashland_20190919_TM_geo.pdf'),
    (42.375, -74.25, 'NY_Hensonville_20190919_TM_geo.pdf'),
    (42.375, -74.125, 'NY_Freehold_20190919_TM_geo.pdf'),
    (42.375, -74.0, 'NY_Leeds_20190926_TM_geo.pdf'),
    (42.375, -73.875, 'NY_Hudson_North_20190919_TM_geo.pdf'),
    (42.375, -73.75, 'NY_Stottville_20190919_TM_geo.pdf'),
    (42.375, -73.625, 'NY_Chatham_20190919_TM_geo.pdf'),
    (42.375, -73.5, 'MA_State_Line_20210415_TM_geo.pdf'),

    (42.25, -75.125, 'NY_Walton_East_20190925_TM_geo.pdf'),
    (42.25, -75.0, 'NY_Hamden_20160818_TM_geo.pdf'),
    (42.25, -74.875, 'NY_Andes_20190919_TM_geo.pdf'),
    (42.25, -74.75, 'NY_Margaretville_20190919_TM_geo.pdf'),
    (42.25, -74.625, 'NY_Fleischmanns_20190925_TM_geo.pdf'),
    (42.25, -74.5, 'NY_West_Kill_20190919_TM_geo.pdf'),
    (42.25, -74.375, 'NY_Lexington_20190919_TM_geo.pdf'),
    (42.25, -74.25, 'NY_Hunter_20190919_TM_geo.pdf'),
    (42.25, -74.125, 'NY_Kaaterskill_Clove_20190919_TM_geo.pdf'),
    (42.25, -74.0, 'NY_Cementon_20190919_TM_geo.pdf'),
    (42.25, -73.875, 'NY_Hudson_South_20190919_TM_geo.pdf'),
    (42.25, -73.75, 'NY_Claverack_20190919_TM_geo.pdf'),
    (42.25, -73.625, 'NY_Hillsdale_20190926_TM_geo.pdf'),
    (42.25, -73.5, 'MA_Egremont_20210415_TM_geo.pdf'),
    
    (42.125, -75.125, 'NY_Corbett_20190925_TM_geo.pdf'),
    (42.125, -75.0, 'NY_Downsville_20190919_TM_geo.pdf'),
    (42.125, -74.875, 'NY_Lewbeach_20190919_TM_geo.pdf'),
    (42.125, -74.75, 'NY_Arena_20190919_TM_geo.pdf'),
    (42.125, -74.625, 'NY_Seager_20190919_TM_geo.pdf'),
    (42.125, -74.5, 'NY_Shandaken_20190919_TM_geo.pdf'),
    (42.125, -74.375, 'NY_Phoenicia_20190919_TM_geo.pdf'),
    (42.125, -74.25, 'NY_Bearsville_20190919_TM_geo.pdf'),
    (42.125, -74.125, 'NY_Woodstock_20190919_TM_geo.pdf'),
    (42.125, -74.0, 'NY_Saugerties_20190919_TM_geo.pdf'),
    (42.125, -73.875, 'NY_Clermont_20190919_TM_geo.pdf'),
    (42.125, -73.75, 'NY_Ancram_20190919_TM_geo.pdf'),
    (42.125, -73.625, 'NY_Copake_20190919_TM_geo.pdf'),
    (42.125, -73.5, 'MA_Bash_Bish_Falls_20210415_TM_geo.pdf'),

    (42.0, -75.125, 'NY_Horton_20190916_TM_geo.pdf'),
    (42.0, -75.0, 'NY_Roscoe_20190916_TM_geo.pdf'),
    (42.0, -74.875, 'NY_Livingston_Manor_20190916_TM_geo.pdf'),
    (42.0, -74.75, 'NY_Willowemoc_20190916_TM_geo.pdf'),
    (42.0, -74.625, 'NY_Claryville_20190916_TM_geo.pdf'),
    (42.0, -74.5, 'NY_Peekamoose_Mountain_20190916_TM_geo.pdf'),
    (42.0, -74.375, 'NY_West_Shokan_20190916_TM_geo.pdf'),
    (42.0, -74.25, 'NY_Ashokan_20190916_TM_geo.pdf'),
    (42.0, -74.125, 'NY_Kingston_West_20190916_TM_geo.pdf'),
    (42.0, -74.0, 'NY_Kingston_East_20190930_TM_geo.pdf'),
    (42.0, -73.875, 'NY_Rock_City_20190930_TM_geo.pdf'),
    (42.0, -73.75, 'NY_Pine_Plains_20190927_TM_geo.pdf'),
    (42.0, -73.625, 'NY_Millerton_20190927_TM_geo.pdf'),
    (42.0, -73.5, 'CT_Sharon_20210227_TM_geo.pdf'),

    (41.875, -75.125, 'NY_Callicoon_20190920_TM_geo.pdf'),
    (41.875, -75.0, 'NY_Jeffersonville_20190923_TM_geo.pdf'),
    (41.875, -74.875, 'NY_Liberty_West_20190916_TM_geo.pdf'),
    (41.875, -74.75, 'NY_Liberty_East_20190923_TM_geo.pdf'),
    (41.875, -74.625, 'NY_Grahamsville_20190916_TM_geo.pdf'),
    (41.875, -74.5, 'NY_Rondout_Reservoir_20190916_TM_geo.pdf'),
    (41.875, -74.375, 'NY_Kerhonkson_20190916_TM_geo.pdf'),
    (41.875, -74.25, 'NY_Mohonk_Lake_20190916_TM_geo.pdf'),
    (41.875, -74.125, 'NY_Rosendale_20190923_TM_geo.pdf'),
    (41.875, -74.0, 'NY_Hyde_Park_20190927_TM_geo.pdf'),
    (41.875, -73.875, 'NY_Salt_Point_20190927_TM_geo.pdf'),
    (41.875, -73.75, 'NY_Millbrook_20190926_TM_geo.pdf'),
    (41.875, -73.625, 'NY_Amenia_20190926_TM_geo.pdf'),
    (41.875, -73.5, 'CT_Ellsworth_20210305_TM_geo.pdf'),

    (41.75, -75.125, 'PA_Damascus_20190904_TM_geo.pdf'),
    (41.75, -75.0, 'NY_Lake_Huntington_20190916_TM_geo.pdf'), 
    (41.75, -74.875, 'NY_White_Lake_20190916_TM_geo.pdf'), 
    (41.75, -74.75, 'NY_Monticello_20190923_TM_geo.pdf'), 
    (41.75, -74.625, 'NY_Woodridge_20190923_TM_geo.pdf'), 
    (41.75, -74.5, 'NY_Ellenville_20190916_TM_geo.pdf'), 
    (41.75, -74.375, 'NY_Napanoch_20190916_TM_geo.pdf'), 
    (41.75, -74.25, 'NY_Gardiner_20190916_TM_geo.pdf'), 
    (41.75, -74.125, 'NY_Clintondale_20190916_TM_geo.pdf'), 
    (41.75, -74.0, 'NY_Poughkeepsie_20190927_TM_geo.pdf'), 
    (41.75, -73.875, 'NY_Pleasant_Valley_20190926_TM_geo.pdf'), 
    (41.75, -73.75, 'NY_Verbank_20190926_TM_geo.pdf'),  
    (41.75, -73.625, 'NY_Dover_Plains_20190930_TM_geo.pdf'), 
    (41.75, -73.5, 'CT_Kent_20210305_TM_geo.pdf'),
]

In [ ]:
FILENAMES = [m[2] for m in map_corners]


def filename_to_url(filename):
    state = filename[:2]
    return f"https://prd-tnm.s3.amazonaws.com/StagedProducts/Maps/USTopo/PDF/{state}/{filename}"

def download_file(filename):
    print(f"Downloading {filename}")
    outdir = "./maps"
    os.makedirs(outdir, exist_ok=True)

    if os.path.exists(os.path.join(outdir, filename)):
        return True
    url = filename_to_url(filename)
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(outdir, filename), 'wb') as f:
            f.write(response.content)
        return True
    else:
        return False

with ThreadPoolExecutor(max_workers=64) as executor:
    for _ in executor.map(download_file, FILENAMES):
        pass


In [ ]:
def get_corners_from_gdal_ds(gdal_ds):
    ulx, xres, _, lry, _, yres = gdal_ds.GetGeoTransform()
    uly = orig_pdf.RasterYSize * yres + lry
    lrx = orig_pdf.RasterXSize * xres + ulx
    return ulx, uly, lrx, lry

# Get approx aspect ratio - just pick random one from the middle
fn = map_corners[len(map_corners)//2][-1]
orig_pdf = gdal.Open(os.path.join('maps', fn))
ulx, uly, lrx, lry = get_corners_from_gdal_ds(orig_pdf)
ASPECT_RATIO = (ulx - lrx) / (uly - lry)

def map_filename(quadrangle):
    return os.path.join('maps', quadrangle)

def warped_filename(quadrangle):
    return os.path.join('warped', quadrangle + '.tiff')

def corrected_filename(quadrangle):
    return os.path.join('corrected', quadrangle + '.tiff')

def warp_pdf(map_info):
    os.makedirs('warped', exist_ok=True)
    uly, ulx, quadrangle = map_info
    out_path = warped_filename(quadrangle)

    if os.path.exists(out_path):
        return gdal.Open(out_path)

    orig_pdf = gdal.Open(map_filename(quadrangle))
    target_size = 4096 # equidistant
    warped = gdal.Warp(
        out_path, orig_pdf, 
        dstSRS='EPSG:4326', # WGS84 
        resampleAlg='cubic',
        width=target_size, height=target_size,
        # this takes min_x, min_y, max_x, max_y
        outputBounds=(ulx, uly - 0.125, ulx + 0.125, uly)
    );
    warped.FlushCache();
    return warped

def gdal_ds_to_image(gdal_ds):
    a1 = gdal_ds.GetRasterBand(1).ReadAsArray()
    a2 = gdal_ds.GetRasterBand(2).ReadAsArray()
    a3 = gdal_ds.GetRasterBand(3).ReadAsArray()

    arr = np.stack((a1, a2, a3), axis=2)
    return Image.fromarray(arr)

def correct_aspect_ratio(im):
    return im.resize(
        (int(im.width * ASPECT_RATIO), im.height), Image.Resampling.BICUBIC
    )

def do_all_the_things(map_info):
    os.makedirs('corrected', exist_ok=True)
    out_path = corrected_filename(map_info[-1])
    if os.path.exists(out_path):
        return
    
    im = correct_aspect_ratio(gdal_ds_to_image(warp_pdf(map_info)))
    im.save(out_path)


In [ ]:
with ThreadPoolExecutor(max_workers=64) as executor:
    for _ in executor.map(do_all_the_things, map_corners):
        pass

In [ ]:
def merge_files(map_corners, crop_pixels=(0, 0, 0, 0)):
    
    row_lats = list(set(m[0] for m in map_corners))
    row_lats.sort()
    column_lons = list(set(m[1] for m in map_corners))
    column_lons.sort()

    # n.b. all input files have been transformed to the same size
    for ir, row in enumerate(row_lats):
        print(row)
        maps = [m for m in map_corners if m[0] == row]
        print(', '.join(m[-1] for m in maps))

        for ic, col in enumerate(column_lons):
            m = [m for m in maps if m[1] == col][0][-1]
            if ic == 0:
                im = Image.open(corrected_filename(m))
                im = crop_image(im, crop_pixels)
                continue 
            im_new = Image.open(corrected_filename(m))
            im_new = crop_image(im_new, crop_pixels)
            im = merge_im_horizontally(im, im_new)
        
        if ir == 0:
            im_all = im
            continue
        im_all = merge_im_vertically(im_all, im)

    return im_all

def crop_image(im, n_pixels):
    box = (n_pixels[0], n_pixels[1], im.width - n_pixels[2], im.height - n_pixels[3])
    return im.crop(box)
            
def merge_im_horizontally(im1, im2):
    w = im1.width + im2.width 
    h = im1.height
    im = Image.new("RGB", (w, h))
    im.paste(im1)
    im.paste(im2, (im1.width, 0))
    return im

def merge_im_vertically(im1, im2):
    w = im1.width
    h = im1.height + im2.height 
    im = Image.new("RGB", (w, h))
    im.paste(im2)
    im.paste(im1, (0, im2.height))
    return im

def increase_contrast(im, contrast_scale=3):
    enhancer = ImageEnhance.Contrast(im)
    return enhancer.enhance(contrast_scale)

In [ ]:
im = merge_files(map_corners, (2, 1, 1, 2))
im.save('test.tiff')
im3 = increase_contrast(im, 1.5)

In [ ]:
Image.MAX_IMAGE_PIXELS = 1110709224 + 1
ASPECT = 17.5 / 9.5
CROPPED_HEIGHT = 22450
X_OFFSET = 3500
Y_OFFSET = 2000
im3.crop((X_OFFSET, Y_OFFSET, X_OFFSET + int(ASPECT * CROPPED_HEIGHT), Y_OFFSET + CROPPED_HEIGHT)).save('final_map.tiff')